In [1253]:
# Entrega II cadeira Web-semantica @ Fcul 2020-2021
# Realizado por João Pinto(55018) e Ricardo Meireles(xxxxx) 

In [1254]:
import csv
import rdflib
import re
from rdflib import Graph, Literal, Namespace, URIRef, BNode
from rdflib.plugins.sparql import prepareQuery
from rdflib.namespace import DCTERMS, RDF, RDFS, XSD
from urllib.parse import quote
from SPARQLWrapper import SPARQLWrapper, XML, TSV, POST, DIGEST
from string import Template


ListaEquipas = ["Equipa"]

NFL = Namespace('http://www.nfl.com/')
pff = Namespace('http://www.pff.com/')

Schema = Namespace("http://schema.org/")
Schema.Person = rdflib.term.URIRef(u'http://schema.org/Person')
Schema.City = rdflib.term.URIRef(u'http://schema.org/City')
Schema.SportsTeam = rdflib.term.URIRef(u'http://schema.org/SportsTeam')
Schema.Athlete = rdflib.term.URIRef(u'http://schema.org/athlete')
Schema.CollegeOrUniversity = rdflib.term.URIRef(u'http://schema.org/CollegeOrUniversity')

statid=1

dados_equipas = csv.DictReader(open("nfl_teams.csv"))
dados_base = csv.DictReader(open("Basic_Stats_Sample.csv"))
dados_passing = csv.DictReader(open("Career_Stats_Passing_Sample.csv"))


dados = Graph()

In [1255]:
for row in dados_equipas:
    idequipa =row['ID']
    nomeequipa = row ['Name']
    equipa=URIRef(NFL+idequipa)
    
    ListaEquipas.append(nomeequipa)

    dados.add((equipa, RDF.type, Schema.SportsTeam))
    dados.add((equipa, RDFS.label, Literal(nomeequipa,lang="en")))
    dados.add((equipa,Schema.name,Literal(nomeequipa)))
    dados.add((equipa,Schema.sport,Literal("American Football")))
    
    
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.addDefaultGraph("http://dbpedia.org")
    query =Template( """
    PREFIX dbpedia: <http://dbpedia.org/resource/> 
    Select ?team ?coach ?owner ?fd ?sb ?abs
     WHERE{ 
        ?team rdfs:label "$nome_equipa"@en;
        dbo:coach ?coach;
        dbo:owner ?owner;
        dbo:formationDate ?fd;
        dbp:noSbChamps ?sb;
        dbo:abstract ?abs.
        FILTER (lang(?abs) = 'en')
    }
        LIMIT 1
    """)


    sparql.setQuery(query.substitute(nome_equipa=nomeequipa))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    for result in results["results"]["bindings"]:
     coach = str(result["coach"]["value"]) 
     owner = str(result["owner"]["value"])
     abs = str(result["abs"]["value"]) 
     fd = str(result["fd"]["value"])
     sb = int(result["sb"]["value"])   
    
        
    dados.add((equipa,Schema.coach,URIRef(coach)))
    dados.add((equipa,Schema.owner,URIRef(owner)))
    dados.add((equipa,Schema.noSbChamps ,Literal(sb,datatype=XSD.int)))  
    dados.add((equipa,Schema.abstract  ,Literal(abs))) 
    dados.add((equipa,Schema.formationDate  ,Literal(fd,datatype=XSD.date))) 


In [1256]:
for row in dados_base:
    
    id =row['Player Id'] 

    current_team = row['Current Team']

    name = Literal(row['Name'])
    position = Literal(row['Position'])
    player_number = Literal(row['Number'],datatype=XSD.int)
    current_status = Literal(row['Current Status'])
    experience = Literal(row['Experience'])
    birth_place = row['Birth Place']
    birth_day = Literal(row['Birthday'],datatype=XSD.date)
    college = Literal(row['College'])
    high_school = Literal(row['High School'])
    height = Literal(row['Height (inches)'],datatype=XSD.int)
    weight = Literal(row['Weight (lbs)'],datatype=XSD.int)
    year_played = Literal(row['Years Played'])

    #tratamento do player para tirar o nome ao id e addicionar como person e atleta
    
    string_player = str(id)
    player = URIRef(NFL+string_player.split('/')[1])

    dados.add((player, RDF.type, Schema.Person))
    dados.add((player, RDF.type, Schema.Athlete))
    
    dados.add((player, Schema.Gender,  URIRef("http://schema.org/Male") ))

    
    #addicionar birthday ao schema
    dados.add((player,Schema.birthDate,birth_day))

    # Tratamento do campo name, separar por nome e nome de familia e addicionar o triplo com o schema
    string_name = str(name)
    dados.add((player, Schema.givenName, Literal(string_name.split(',')[1] )))  
    dados.add((player, Schema.familyName, Literal(string_name.split(',')[0] )))  
    
    # Tratamento do years played, separar por temporada e addicionar todas ao triplo
    string_years = str(year_played)
    if  string_years:  
     nums = [int(n) for n in string_years.split('-')] 
    x = range(nums[0], nums[1]+1)
    for n in x: 
     dados.add((player, NFL.yearPlayed, Literal(n,datatype=XSD.int)))  

    # Addicionar jogador como atlete do schema Sports Team
    if current_team:  
      dados.add((URIRef(NFL+str(ListaEquipas.index(current_team))),Schema.atlete,player))  
    
    #tratar a experience e addicionar ao schema
    
    string_exp = row['Experience']
    if string_exp:
     temp = re.findall(r'\d+', string_exp) 
     res = list(map(int, temp)) 
     dados.add((player, NFL.experience, Literal(res[0],datatype=XSD.int)))
    
    #Tratamento do campo Birthplace, retirar estado e addicionar o triplo com o schema
    if birth_place: 
     string_birth = str(birth_place)
     dados.add((player, Schema.birthPlace, Literal(string_birth.split(',')[0] ))) 
        
    dados.add((player, NFL.currentStatus, current_status)) #addicionar current status
    
    dados.add((player, NFL.college, college)) #addicionar college
    
    #addicionar Hight and Weight
    dados.add((player, Schema.height, height))
    dados.add((player, Schema.weight, weight))
    
    if position:   #addicionar posiçao se existir no ficheiro
        dados.add((player, NFL.position, position))
    if player_number:  #addicionar numero do jogador se existir no ficheiro
        dados.add((player, NFL.playerNumber, player_number))
        
    if high_school: #tratamento de high school (retirar o hs) e addicionar ao triploe
        hs_string = high_school
        dados.add((player, NFL.highSchool, Literal(hs_string.split('HS')[0]))) 
        
    dados.add((player, RDFS.label, Literal(nomeequipa,lang="en")))    
        
  

In [1257]:
# for row in dados_passing:
    

    
#     stat = Literal(row['year'])
    
    
#         sid= URIRef(sls+s["sid"])
#     buyer= URIRef(aff+s["buyer"])
#     seller= URIRef(aff+s["seller"])
#     product= URIRef(sls+s["product"])
#     quant=Literal(s["quantity"],datatype=XSD.int)
#     cost=Literal(s["cost"],datatype=XSD.real)

#     G.add((sid, RDF.type, sls.Sale))
#     G.add((sid, sls.Buyer, buyer))
#     G.add((sid, sls.Seller, seller))
#     G.add((sid, sls.Product, product))
#     G.add((sid, sls.Quantity, quant))
#     G.add((sid, sls.Cost, cost))
    
#     dados.add(stat, RDF.type, pff.stat)
    
#     id =row['Player Id'] 
    
#     player = URIRef(NFL+id)

# #     year = Literal(row['Year'])
    
# #     playerseason= URIRef(NFL+id+"/"+quote(row['Year']))
    
#     name = URIRef(NFL+quote(row['Name']))
#     team = URIRef(NFL+quote(row['Team']))
#     games_played = Literal(row['Games Played'])
#     passes_attempted = Literal(row['Passes Attempted'])
#     passes_completed = Literal(row['Passes Completed'])
#     completion_percentage = Literal(row['Completion Percentage'])
#     pass_att_per_game = Literal(row['Pass Attempts Per Game'])
#     passing_yards = Literal(row['Passing Yards'])
    
#     passing_yards_att = Literal(row['Passing Yards Per Attempt'])
#     passing_yards_game = Literal(row['Passing Yards Per Game'])
#     td_passes = Literal(row['TD Passes'])
#     percentage_tds_per_attemp = Literal(row['Percentage of TDs per Attempts'])
    
#     interceptions = Literal(row['Ints'])
#     interceptions_rate = Literal(row['Ints'])
#     longest_pass = Literal(row['Longest Pass'])
#     passes_longer_20yards = Literal(row['Passes Longer than 20 Yards'])
    
#     passes_longer_40yards = Literal(row['Passes Longer than 40 Yards'])
#     sacks = Literal(row['Sacks'])
#     sacked_yards_lost = Literal(row['Sacked Yards Lost'])
#     passer_rating = Literal(row['Passer Rating'])
  

#     dados.add((playerseason, RDF.type, NFL.playerseason))
#     dados.add((playerseason, RDFS.label, Literal(year, lang="en")))

#     dados.add((player, NFL.playerseason, playerseason))

#     dados.add((playerseason,NFL.team,team))
#     dados.add((playerseason,NFL.games_played,games_played))
#     dados.add((playerseason,NFL.passes_attempted,passes_attempted))
#     dados.add((playerseason,NFL.passes_completed,passes_completed))
#     dados.add((playerseason,NFL.completion_percentage,completion_percentage))
#     dados.add((playerseason,NFL.pass_att_per_game,pass_att_per_game))
#     dados.add((playerseason,NFL.passing_yards,passing_yards))
    
#     dados.add((playerseason,NFL.passing_yards_att,passing_yards_att))
#     dados.add((playerseason,NFL.passing_yards_game,passing_yards_game))
#     dados.add((playerseason,NFL.td_passes,td_passes))
#     dados.add((playerseason,NFL.percentage_tds_per_attemp,percentage_tds_per_attemp))
    
#     dados.add((playerseason,NFL.interceptions,interceptions))
#     dados.add((playerseason,NFL.interceptions_rate,interceptions_rate))
#     dados.add((playerseason,NFL.longest_pass,longest_pass))
#     dados.add((playerseason,NFL.passes_longer_20yards,passes_longer_20yards))
    
#     dados.add((playerseason,NFL.passes_longer_40yards,passes_longer_40yards))
#     dados.add((playerseason,NFL.sacks,sacks))
#     dados.add((playerseason,NFL.sacked_yards_lost,sacked_yards_lost))
#     dados.add((playerseason,NFL.passer_rating,passer_rating))
    
#     dados.add((playerseason, NFL.name, name ))  
#     dados.add((playerseason, NFL.year, year ))  


#     dados.add((player, NFL.player_stat, player_stat ))  
#     dados.add((player_stat, year, stat_year ))
    
#     dados.add((stat_year,NFL.team,team))
#     dados.add((stat_year,NFL.games_played,games_played))
#     dados.add((stat_year,NFL.passes_attempted,passes_attempted))
#     dados.add((stat_year,NFL.passes_completed,passes_completed))
#     dados.add((stat_year,NFL.completion_percentage,completion_percentage))
#     dados.add((stat_year,NFL.pass_att_per_game,pass_att_per_game))
#     dados.add((stat_year,NFL.passing_yards,passing_yards))
    
#     dados.add((stat_year,NFL.passing_yards_att,passing_yards_att))
#     dados.add((stat_year,NFL.passing_yards_game,passing_yards_game))
#     dados.add((stat_year,NFL.td_passes,td_passes))
#     dados.add((stat_year,NFL.percentage_tds_per_attemp,percentage_tds_per_attemp))
    
#     dados.add((stat_year,NFL.interceptions,interceptions))
#     dados.add((stat_year,NFL.interceptions_rate,interceptions_rate))
#     dados.add((stat_year,NFL.longest_pass,longest_pass))
#     dados.add((stat_year,NFL.passes_longer_20yards,passes_longer_20yards))
    
#     dados.add((stat_year,NFL.passes_longer_40yards,passes_longer_40yards))
#     dados.add((stat_year,NFL.sacks,sacks))
#     dados.add((stat_year,NFL.sacked_yards_lost,sacked_yards_lost))
#     dados.add((stat_year,NFL.passer_rating,passer_rating))
    

In [1258]:
# for row in dados_passing:
    
#     stat = URIRef(pff+str(statid))
#     statid= statid+1
    
#     player = URIRef(NFL+id)
#     name = URIRef(NFL+quote(row['Name']))
#     team = URIRef(NFL+quote(row['Team']))
#     games_played = Literal(row['Games Played'])
#     passes_attempted = Literal(row['Passes Attempted'])
#     passes_completed = Literal(row['Passes Completed'])
#     completion_percentage = Literal(row['Completion Percentage'])
#     pass_att_per_game = Literal(row['Pass Attempts Per Game'])
#     passing_yards = Literal(row['Passing Yards'])
    
#     passing_yards_att = Literal(row['Passing Yards Per Attempt'])
#     passing_yards_game = Literal(row['Passing Yards Per Game'])
#     td_passes = Literal(row['TD Passes'])
#     percentage_tds_per_attemp = Literal(row['Percentage of TDs per Attempts'])
    
#     interceptions = Literal(row['Ints'])
#     interceptions_rate = Literal(row['Ints'])
#     longest_pass = Literal(row['Longest Pass'])
#     passes_longer_20yards = Literal(row['Passes Longer than 20 Yards'])
    
#     passes_longer_40yards = Literal(row['Passes Longer than 40 Yards'])
#     sacks = Literal(row['Sacks'])
#     sacked_yards_lost = Literal(row['Sacked Yards Lost'])
#     passer_rating = Literal(row['Passer Rating'])
    
#     dados.add((stat, RDF.type, pff.stat))
#     dados.add((stat, NFL.name, name))
#     dados.add((stat, NFL.player, player))
#     dados.add((stat,pff.passing_yards,passing_yards))
    
#     dados.add((stat,NFL.team,team))
#     dados.add((stat,pff.games_played,games_played))
#     dados.add((stat,pff.passes_attempted,passes_attempted))
#     dados.add((stat,pff.passes_completed,passes_completed))
#     dados.add((stat,pff.completion_percentage,completion_percentage))
#     dados.add((stat,pff.pass_att_per_game,pass_att_per_game))
#     dados.add((stat,pff.passing_yards,passing_yards))
    
#     dados.add((stat,pff.passing_yards_att,passing_yards_att))
#     dados.add((stat,pff.passing_yards_game,passing_yards_game))
#     dados.add((stat,pff.td_passes,td_passes))
#     dados.add((stat,pff.percentage_tds_per_attemp,percentage_tds_per_attemp))
    
#     dados.add((stat,pff.interceptions,interceptions))
#     dados.add((stat,pff.interceptions_rate,interceptions_rate))
#     dados.add((stat,pff.longest_pass,longest_pass))
#     dados.add((stat,pff.passes_longer_20yards,passes_longer_20yards))
    
#     dados.add((stat,pff.passes_longer_40yards,passes_longer_40yards))
#     dados.add((stat,pff.sacks,sacks))
#     dados.add((stat,pff.sacked_yards_lost,sacked_yards_lost))
#     dados.add((stat,pff.passer_rating,passer_rating))

In [1259]:
dados.bind("NFL",NFL)
dados.bind("Pro football Focus", pff)
dados.bind("Schema", Schema)

In [1260]:
print(dados.serialize(format="turtle").decode("utf-8"))

@prefix NFL: <http://www.nfl.com/> .
@prefix Schema: <http://schema.org/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

NFL:1 a Schema:SportsTeam ;
    rdfs:label "Arizona Cardinals"@en ;
    Schema:abstract "The Pittsburgh Steelers are a professional American football team based in Pittsburgh, Pennsylvania. The Steelers compete in the National Football League (NFL), as a member club of the league's American Football Conference (AFC) North division. Founded in 1933, the Steelers are the oldest franchise in the AFC. In contrast with their status as perennial also-rans in the pre-merger NFL, where they were the oldest team to never win a league championship, the Steelers of the post-merger (modern) era are one of the most successful NFL franchises. Pittsburgh has won more Super Bowl titles (6), and hosted more (11) conference championship games than any other NFL team. The Steelers share the record for most AFC championships

In [1261]:
# qres = dados.query(
#     """SELECT ?id
#        WHERE {
#           ?p a Schema:SportsTeam .
#           ?p NFL:id ?id.
#           ?p Schema:name "Hoston Texans"
#        }LIMIT 50""")
# for row in qres:
#     print("%s " % row)

In [1262]:
#     q = prepareQuery(
#      """SELECT ?id
#        WHERE {
#           ?p a Schema:SportsTeam .
#           ?p NFL:id ?id.
#           ?p Schema:name ?name
#        }LIMIT 50""")

#     for row in dados.query(q, initBindings={"name": "Houston Texans"}):
#      print(row)

In [1263]:
equipa = "New England Patriots"
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.addDefaultGraph("http://dbpedia.org")
query =Template( """
    PREFIX dbpedia: <http://dbpedia.org/resource/> 
    Select ?team ?coach
     WHERE{ 
        ?team rdfs:label "$nome_equipa"@en;
        dbo:coach ?coach.
    }
""")


sparql.setQuery(query.substitute(uri="New England Patriots"))
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    coach = str(result["coach"]["value"])
    print(coach)


KeyError: 'nome_equipa'